## Import dependancies

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.3 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#for import the data
import pandas as pd
import numpy as np

# for the visualisation
import matplotlib.pyplot as plt
import seaborn as sns

#for the text pre-processing (text cleaning)
import nltk
nltk.download('punkt')
import re #regular expression
import string
from nltk.tokenize import word_tokenize # word tokenization
from nltk.stem import PorterStemmer # word stemming

#for feature extraction ( vectorazation)
from sklearn.feature_extraction.text import CountVectorizer

#for split the data into train and test
from sklearn.model_selection import train_test_split

#for the model training
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

#for the model evaluation
from sklearn.metrics import multilabel_confusion_matrix,classification_report


from nltk.corpus import stopwords
from wordcloud import WordCloud
import contractions
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Import and preprocess dataset

In [ ]:
#Data with negative sample (without annotation)
data_N = pd.read_csv('/content/drive/MyDrive/post classification/processed_data/posts_with_Label_and_without_preflable.csv')

In [ ]:
#Data with ontology and manual label
df = pd.read_csv('/content/drive/MyDrive/post classification/processed_data/posts-ontology-manual-label.csv')

In [ ]:
df.shape

(9497, 4)

In [ ]:
data_N.head()

In [ ]:
df.head()

In [ ]:
# defne new column named 'email_length' that contain the length of each email in the dataset
data_N["post_length"]=data_N["post"].apply(len)

In [ ]:
print('the max length post is:',data_N['post_length'].max())
print('the min length post is:',data_N['post_length'].min())
print('the avg length post is:',data_N['post_length'].mean())

the max length post is: 9985
the min length post is: 1
the avg length post is: 1145.9829419816783


In [ ]:
# delete all emails that its length not in [2500:10]
data_N = data_N[data_N.post_length<2000]
data_N = data_N[data_N.post_length>6]

In [ ]:
print(len(data_N))

7959


In [ ]:
# Check for NaN values and replace them with an empty string
df['post'] = df['post'].fillna('')

# Define a new column named 'post_length' that contains the length of each post in the dataset
df['post_length'] = df['post'].apply(len)

In [ ]:
print('the max length post is:',df['post_length'].max())
print('the min length post is:',df['post_length'].min())
print('the avg length post is:',df['post_length'].mean())

the max length post is: 10013
the min length post is: 0
the avg length post is: 1139.6069427771108


In [ ]:
import re
#define the process of text cleaning
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)
#Clean Text
def clean_text(data):
    # convert catacter to lowercase
    data['clean_text']=data['post'].str.lower()
    #remove URLS
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"http\S+", "", elem))
    #remove ponctuation
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"[^\w\s]", "", elem))
    #remove
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'/n',"",elem))
    #remove degits
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\d+',"",elem))
    #remove emojis
    data['clean_text'] = data['clean_text'].apply(lambda elem:deEmojify(elem))
    #remove multiple spaces
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+'," ",elem))
    #remove single caracter
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+[a-zA-Z]\s+'," ",elem))
    return data


def process_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Handle contractions using the contractions library
    expanded_text = contractions.fix(text)

    # Lowercasing
    expanded_text = expanded_text.lower()

    # Tokenization
    tokens = word_tokenize(expanded_text)

    # Removing Punctuation
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stop Words
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [ ]:
process_df = df.copy()
process_df

In [ ]:
#apply the process of cleaning for the train and test data
process_df = clean_text(process_df)
process_df['post'] = process_df['post'].apply(process_text)
#process_df['clean_text'] = process_df['clean_text'].apply(process_text)



In [ ]:
process_df.head()

In [ ]:
# Assuming 'process_df' is the DataFrame name
# Print the number of posts
print(f"Number of posts: {len(process_df)}")

# Print the count of '1' in the 'obsession' column
print(f"Number of '1's in 'obsession' column: {process_df['obsession'].sum()}")
print(f"Number of '1's in 'compulsion' column: {process_df['compulsion'].sum()}")

# Print the count of '0's in both 'obsession' and 'compulsion' columns
print(f"Number of '0's in 'obsession' column: {len(process_df) - process_df['obsession'].sum()}")
print(f"Number of '0's in 'compulsion' column: {len(process_df) - process_df['compulsion'].sum()}")


Number of posts: 9996
Number of '1's in 'obsession' column: 5029
Number of '1's in 'compulsion' column: 2702
Number of '0's in 'obsession' column: 4967
Number of '0's in 'compulsion' column: 7294


## BOW and splitting the data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Creating features using Bag of Words
text_data = process_df['post'].astype(str)

# Convert text to vectors using CountVectorizer with specified parameters
count_vect = CountVectorizer(max_features=25000, min_df=5, max_df=0.9, stop_words='english')
X = count_vect.fit_transform(text_data)

# Assuming 'compulsion' and 'obsession' are the target labels
Y = np.asarray(process_df[['compulsion', 'obsession']])


In [ ]:
# Get the shape (size) of the DTM
num_documents, num_tokens = X.shape

# Print the number of documents and tokens
print("Number of documents:", num_documents)
print("Number of unique tokens:", num_tokens)

Number of documents: 9996
Number of unique tokens: 7281


In [ ]:

print(X.shape)
print(Y.shape)

(9996, 7281)
(9996, 2)


In [ ]:
# Assuming X and Y are your feature and target variables
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7996, 7281)
(2000, 7281)
(7996, 2)
(2000, 2)


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

# Assuming X_train, X_test, y_train, y_test are your training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_classifier.predict(X_test)

# Evaluate the model
conf_matrix = multilabel_confusion_matrix(y_test, y_pred_rf)

# Print confusion matrix for each class
for i, label in enumerate(['obsession', 'compulsion']):
    print(f"Confusion Matrix for {label}:")
    print(conf_matrix[i])
    print("\n")


Confusion Matrix for obsession:
[[1419    3]
 [ 173  405]]


Confusion Matrix for compulsion:
[[845 151]
 [ 15 989]]




In [ ]:
# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], y_pred_rf[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')


Accuracy for obsession: 0.912
Accuracy for compulsion: 0.917


In [ ]:
#classification report
label_names = ['obsession','compulsion']
report = classification_report(y_test, y_pred_rf, target_names=label_names)
print(report)

              precision    recall  f1-score   support

   obsession       0.99      0.70      0.82       578
  compulsion       0.87      0.99      0.92      1004

   micro avg       0.90      0.88      0.89      1582
   macro avg       0.93      0.84      0.87      1582
weighted avg       0.91      0.88      0.89      1582
 samples avg       0.61      0.61      0.61      1582



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Prediction on manual label data

In [ ]:
unlabed_data = pd.read_csv('/content/drive/MyDrive/post classification/manual_check/manual-check-small-data.csv')

In [ ]:
unlabed_data.head()

In [ ]:
#import model
import joblib
#clf = joblib.load('../model/RFModel.pkl')
LR_BOW = joblib.load("/content/drive/MyDrive/post classification/model/ml_model/LR_BOW.pkl")

In [ ]:
#apply the process of cleaning for the train and test data
unlabed_data = clean_text(unlabed_data)
unlabed_data['post'] = unlabed_data['post'].apply(process_text)
#process_df['clean_text'] = process_df['clean_text'].apply(process_text)
unlabed_data.head()

In [ ]:
# Assuming 'clean_text' is the column containing text data
unlabeled_data_text = unlabed_data['post']

# Transform the unlabelled data using the same CountVectorizer
X1 = count_vect.transform(unlabeled_data_text)

# Make predictions using the loaded model
predictions = clf_LR.predict(X1)

In [ ]:
# Assuming 'predictions' is a NumPy array or Pandas DataFrame containing the predicted labels
# Add these predictions to your unlabeled data or save them as needed
unlabed_data['predicted_compulsion'] = predictions[:, 0]
unlabed_data['predicted_obsession'] = predictions[:, 1]

# Display or save the results
print(unlabed_data[['post', 'predicted_compulsion', 'predicted_obsession']])

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Assuming 'clean_text' is the column containing text data
unlabeled_data_vec_first_470 = count_vect.transform(unlabed_data['post'].head(470))

# Make predictions using the loaded model for the first 470 rows
predictions_first_470 = LR_BOW.predict(unlabeled_data_vec_first_470)

# Assuming unlabeled_data is your DataFrame for the first 470 rows
# Replace '...' with your actual DataFrame variable
unlabeled_data_first_470 = unlabed_data.head(470)

# Replace NaN values in the manual labels with 0 (you can choose a different approach)
y_true_first_470 = unlabeled_data_first_470[['obsession_first_checker', 'compulsion_first_checker']].fillna(0).values

# Evaluate the model on the first 470 rows
print("\nEvaluation for 'obsession':")
print(classification_report(y_true_first_470[:, 0], predictions_first_470[:, 0]))

print("\nEvaluation for 'compulsion':")
print(classification_report(y_true_first_470[:, 1], predictions_first_470[:, 1]))

# Print accuracy
print("\nAccuracy for 'obsession':", accuracy_score(y_true_first_470[:, 0], predictions_first_470[:, 0]))
print("Accuracy for 'compulsion':", accuracy_score(y_true_first_470[:, 1], predictions_first_470[:, 1]))




Evaluation for 'obsession':
              precision    recall  f1-score   support

         0.0       0.64      0.98      0.78       300
         1.0       0.44      0.02      0.04       170

    accuracy                           0.64       470
   macro avg       0.54      0.50      0.41       470
weighted avg       0.57      0.64      0.51       470


Evaluation for 'compulsion':
              precision    recall  f1-score   support

         0.0       0.91      0.85      0.88       420
         1.0       0.18      0.28      0.22        50

    accuracy                           0.79       470
   macro avg       0.55      0.56      0.55       470
weighted avg       0.83      0.79      0.81       470


Accuracy for 'obsession': 0.6361702127659574
Accuracy for 'compulsion': 0.7893617021276595


In [ ]:
unlabed_data.head()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Assuming df is your DataFrame containing the mentioned columns
# Replace this line with your actual DataFrame
df = ...

# Extract the true labels (manual labels) and predicted labels
y_true = unlabed_data[['obsession_first_checker', 'compulsion_first_checker']].values
y_pred = unlabed_data[['predicted_obsession', 'predicted_compulsion']].values

# Evaluate the model on the first 470 posts (exclude rows without manual labels)
mask = ~unlabed_data['obsession_first_checker'].isnull()
y_true_sub = y_true[mask]
y_pred_sub = y_pred[mask]

# Count the number of manual labels for 'obsession'
obsession_manual_counts = unlabed_data['obsession_first_checker'].value_counts().to_dict()
print("\nManual Label Counts for 'obsession':")
print("Label 0:", obsession_manual_counts.get(0, 0))
print("Label 1:", obsession_manual_counts.get(1, 0))

# Count the number of manual labels for 'compulsion'
compulsion_manual_counts = unlabed_data['compulsion_first_checker'].value_counts().to_dict()
print("\nManual Label Counts for 'compulsion':")
print("Label 0:", compulsion_manual_counts.get(0, 0))
print("Label 1:", compulsion_manual_counts.get(1, 0))

# Count the number of predicted labels for 'obsession'
obsession_pred_counts = unlabed_data['predicted_obsession'].value_counts().to_dict()
print("\nPredicted Label Counts for 'obsession':")
print("Label 0:", obsession_pred_counts.get(0, 0))
print("Label 1:", obsession_pred_counts.get(1, 0))

# Count the number of predicted labels for 'compulsion'
compulsion_pred_counts = unlabed_data['predicted_compulsion'].value_counts().to_dict()
print("\nPredicted Label Counts for 'compulsion':")
print("Label 0:", compulsion_pred_counts.get(0, 0))
print("Label 1:", compulsion_pred_counts.get(1, 0))




Manual Label Counts for 'obsession':
Label 0: 321
Label 1: 176

Manual Label Counts for 'compulsion':
Label 0: 446
Label 1: 51

Predicted Label Counts for 'obsession':
Label 0: 447
Label 1: 52

Predicted Label Counts for 'compulsion':
Label 0: 489
Label 1: 10


## NB

In [ ]:
#intialize the Random Forest Model
clf_NB = MultiOutputClassifier(MultinomialNB()).fit(X_train, y_train)
#train the model
clf_NB.fit(X_train,y_train)

MultiOutputClassifier(estimator=MultinomialNB())

In [ ]:
prediction_NB= clf_NB.predict(X_test)

In [ ]:
prediction_NB

array([[1, 0],
       [1, 0],
       [0, 1],
       ...,
       [1, 0],
       [0, 1],
       [0, 0]])

In [ ]:
#classification report
label_names = ['obsession','compulsion']
print(classification_report(y_test, prediction_NB,target_names=label_names))

              precision    recall  f1-score   support

   obsession       0.70      0.79      0.74       578
  compulsion       0.77      0.88      0.82      1004

   micro avg       0.74      0.85      0.79      1582
   macro avg       0.73      0.84      0.78      1582
weighted avg       0.74      0.85      0.79      1582
 samples avg       0.56      0.59      0.56      1582



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# Print confusion matrix
# Print confusion matrix
conf_matrix = multilabel_confusion_matrix(y_test, prediction_NB)
print("Confusion Matrix:")
print(conf_matrix)

#For the 'obsession' class:
#True Positive (TP): 425
#False Positive (FP): 162
#True Negative (TN): 1260
#False Negative (FN): 153

#For the 'compulsion' class:
#True Positive (TP): 917
#False Positive (FP): 349
#True Negative (TN): 647
#False Negative (FN): 87


#True Positive (TP): Instances correctly predicted as belonging to the class.
#False Positive (FP): Instances incorrectly predicted as belonging to the class (they don't actually belong to it).
#True Negative (TN): Instances correctly predicted as not belonging to the class.
#False Negative (FN): Instances incorrectly predicted as not belonging to the class (they actually belong to it).


Confusion Matrix:
[[[1260  162]
  [ 153  425]]

 [[ 647  349]
  [  87  917]]]


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, prediction_NB)
print(f'Accuracy: {accuracy}')


Accuracy: 0.6935


In [ ]:

# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], prediction_NB[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')


Accuracy for obsession: 0.8425
Accuracy for compulsion: 0.808


## Prediction on manual label using NB

In [ ]:
# Assuming 'clean_text' is the column containing text data
unlabeled_data_vec_first_470 = count_vect.transform(unlabed_data['clean_text'].head(470))

# Make predictions using the loaded model for the first 470 rows
predictions_first_470 = clf_NB.predict(unlabeled_data_vec_first_470)

# Assuming unlabeled_data is your DataFrame for the first 470 rows
# Replace '...' with your actual DataFrame variable
unlabeled_data_first_470 = unlabed_data.head(470)

# Replace NaN values in the manual labels with 0 (you can choose a different approach)
y_true_first_470 = unlabeled_data_first_470[['obsession_first_checker', 'compulsion_first_checker']].fillna(0).values

# Evaluate the model on the first 470 rows
print("\nEvaluation for 'obsession':")
print(classification_report(y_true_first_470[:, 0], predictions_first_470[:, 0]))

print("\nEvaluation for 'compulsion':")
print(classification_report(y_true_first_470[:, 1], predictions_first_470[:, 1]))

# Print accuracy
print("\nAccuracy for 'obsession':", accuracy_score(y_true_first_470[:, 0], predictions_first_470[:, 0]))
print("Accuracy for 'compulsion':", accuracy_score(y_true_first_470[:, 1], predictions_first_470[:, 1]))


Evaluation for 'obsession':
              precision    recall  f1-score   support

         0.0       0.64      0.90      0.75       300
         1.0       0.37      0.11      0.16       170

    accuracy                           0.61       470
   macro avg       0.50      0.50      0.46       470
weighted avg       0.54      0.61      0.54       470


Evaluation for 'compulsion':
              precision    recall  f1-score   support

         0.0       0.89      0.39      0.54       420
         1.0       0.10      0.58      0.17        50

    accuracy                           0.41       470
   macro avg       0.49      0.49      0.36       470
weighted avg       0.80      0.41      0.50       470


Accuracy for 'obsession': 0.6106382978723405
Accuracy for 'compulsion': 0.4106382978723404


## K-Nearest Neighbors (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, y_test are your training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the KNN model with specified parameters
knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto')

# Train the KNN model
knn_model.fit(X_train, y_train)

# Make predictions on the test set
knn_predictions = knn_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, knn_predictions)

print(f'Accuracy for KNN: {accuracy}')

Accuracy for KNN: 0.5595


In [ ]:
#classification report
label_names = ['obsession','compulsion']
print(classification_report(y_test, knn_predictions,target_names=label_names))

              precision    recall  f1-score   support

   obsession       0.84      0.20      0.33       578
  compulsion       0.87      0.56      0.68      1004

   micro avg       0.87      0.43      0.57      1582
   macro avg       0.86      0.38      0.50      1582
weighted avg       0.86      0.43      0.55      1582
 samples avg       0.32      0.30      0.30      1582



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# Print confusion matrix
# Print confusion matrix
conf_matrix = multilabel_confusion_matrix(y_test, knn_predictions)
print("Confusion Matrix:")
print(conf_matrix)

#For the 'obsession' class:
#True Positive (TP): 101
#False Positive (FP): 39
#True Negative (TN): 1383
#False Negative (FN): 477

#For the 'compulsion' class:
#True Positive (TP): 511
#False Positive (FP): 75
#True Negative (TN): 921
#False Negative (FN): 493


#True Positive (TP): Instances correctly predicted as belonging to the class.
#False Positive (FP): Instances incorrectly predicted as belonging to the class (they don't actually belong to it).
#True Negative (TN): Instances correctly predicted as not belonging to the class.
#False Negative (FN): Instances incorrectly predicted as not belonging to the class (they actually belong to it).


Confusion Matrix:
[[[1399   23]
  [ 460  118]]

 [[ 916   80]
  [ 445  559]]]


In [ ]:
# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], knn_predictions[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')

Accuracy for obsession: 0.7585
Accuracy for compulsion: 0.7375


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, knn_predictions)
print(f'Accuracy: {accuracy}')


Accuracy: 0.5595


# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer(max_features=3000, max_df=0.9)  # Adjust max_features as needed
X_tfidf = tfidf_vectorizer.fit_transform(process_df['post'])


# Assuming 'compulsion' and 'obsession' are the target labels
y = np.asarray(process_df[['compulsion', 'obsession']])

In [ ]:

# Split the data into training and testing sets
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)


## Logistic regression using tf-idf

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
# Re-fit the MultiOutputClassifier
clf_LR_tf = MultiOutputClassifier(LogisticRegression()).fit(X_train_tfidf, y_train)

# Example prediction
y_pred_tf = clf_LR_tf.predict(X_test_tfidf)


In [ ]:
label_names = ['obsession','compulsion']
print(classification_report(y_test, y_pred_tf,target_names=label_names))

              precision    recall  f1-score   support

   obsession       0.99      0.72      0.83       578
  compulsion       0.92      0.93      0.92      1004

   micro avg       0.94      0.85      0.89      1582
   macro avg       0.95      0.82      0.88      1582
weighted avg       0.94      0.85      0.89      1582
 samples avg       0.60      0.60      0.59      1582



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# Print confusion matrix
# Print confusion matrix
conf_matrix = multilabel_confusion_matrix(y_test, y_pred_tf)
print("Confusion Matrix:")
print(conf_matrix)

#For the 'obsession' class:
#True Positive (TP): 511
#False Positive (FP): 28
#True Negative (TN): 1394
#False Negative (FN): 67

#For the 'compulsion' class:
#True Positive (TP): 968
#False Positive (FP): 35
#True Negative (TN): 961
#False Negative (FN): 36


#True Positive (TP): Instances correctly predicted as belonging to the class.
#False Positive (FP): Instances incorrectly predicted as belonging to the class (they don't actually belong to it).
#True Negative (TN): Instances correctly predicted as not belonging to the class.
#False Negative (FN): Instances incorrectly predicted as not belonging to the class (they actually belong to it).


Confusion Matrix:
[[[1417    5]
  [ 162  416]]

 [[ 911   85]
  [  72  932]]]


In [ ]:
# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], y_pred_tf[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')

Accuracy for obsession: 0.9165
Accuracy for compulsion: 0.9215


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_tf)
print(f'Accuracy: {accuracy}')


Accuracy: 0.854


In [ ]:

# Save the model and TF-IDF vectorizer
joblib.dump(clf_LR_tf, 'model_LR_tf.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [ ]:
# Load the TF-IDF vectorizer
tfidf_vectorizer_loaded = joblib.load('tfidf_vectorizer.pkl')

# Transform the unlabeled data using the loaded TF-IDF vectorizer
unlabeled_data_tf = tfidf_vectorizer_loaded.transform(unlabed_data['clean_text'])

# Load the trained model
clf_LR_tf_loaded = joblib.load('model_LR_tf.pkl')

# Make predictions using the loaded model
predictions = clf_LR_tf_loaded.predict(unlabeled_data_tf)

In [ ]:
# Load the manual labels for 'obsession' and 'compulsion'
y_true_obsession = unlabed_data['obsession_first_checker']
y_true_compulsion = unlabed_data['compulsion_first_checker']

# Drop rows with NaN values in the 'obsession_first_checker' column
y_true_obsession = y_true_obsession.dropna()

# Drop corresponding rows with NaN values in the 'compulsion_first_checker' column
y_true_compulsion = y_true_compulsion.dropna()


# Print the number of samples for investigation
print("Number of samples for 'y_true_obsession':", len(y_true_obsession))
print("Number of samples for 'predictions[:, 0]':", len(predictions[:, 0]))



Number of samples for 'y_true_obsession': 497
Number of samples for 'predictions[:, 0]': 499


In [ ]:
# Load the manual labels for 'obsession' and 'compulsion'
y_true_obsession = unlabed_data['obsession_first_checker']
y_true_compulsion = unlabed_data['compulsion_first_checker']

# Print the number of samples for investigation
print("Number of samples for 'y_true_obsession':", len(y_true_obsession))
print("Number of samples for 'predictions[:, 0]':", len(predictions[:, 0]))

# Drop rows with NaN values in 'obsession_first_checker'
y_true_obsession = y_true_obsession.dropna()
y_true_compulsion = y_true_compulsion.dropna()

# Ensure that 'predictions[:, 0]' has the same length as 'y_true_obsession'
predictions_obsession = predictions[:len(y_true_obsession), 0]
predictions_compulsion= predictions[:len(y_true_compulsion), 0]

# Print the updated number of samples for investigation
print("Updated number of samples for 'y_true_obsession':", len(y_true_obsession))
print("Updated number of samples for 'predictions_obsession':", len(predictions_obsession))


Number of samples for 'y_true_obsession': 499
Number of samples for 'predictions[:, 0]': 499
Updated number of samples for 'y_true_obsession': 497
Updated number of samples for 'predictions_obsession': 497


In [ ]:
from sklearn.metrics import classification_report

# Evaluate the predictions for 'obsession'
print("\nEvaluation for 'obsession':")
print(classification_report(y_true_obsession, predictions_obsession))

print("\nEvaluation for 'compulsion':")
print(classification_report(y_true_compulsion, predictions_compulsion))


Evaluation for 'obsession':
              precision    recall  f1-score   support

         0.0       0.64      0.98      0.78       321
         1.0       0.29      0.01      0.02       176

    accuracy                           0.64       497
   macro avg       0.47      0.50      0.40       497
weighted avg       0.52      0.64      0.51       497


Evaluation for 'compulsion':
              precision    recall  f1-score   support

         0.0       0.90      0.99      0.94       446
         1.0       0.14      0.02      0.03        51

    accuracy                           0.89       497
   macro avg       0.52      0.50      0.49       497
weighted avg       0.82      0.89      0.85       497



## Gradient Boosting using tf-idf:

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Initialize the Gradient Boosting model
gb_model = GradientBoostingClassifier()

# Wrap the model with MultiOutputClassifier
multi_output_gb = MultiOutputClassifier(gb_model)

# Train the MultiOutputClassifier model
multi_output_gb.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_predictions_GB = multi_output_gb.predict(X_test_tfidf)

In [ ]:
label_names = ['obsession','compulsion']
print(classification_report(y_test, y_predictions_GB,target_names=label_names))

              precision    recall  f1-score   support

   obsession       0.99      0.92      0.95       578
  compulsion       0.98      0.97      0.98      1004

   micro avg       0.98      0.95      0.97      1582
   macro avg       0.99      0.95      0.97      1582
weighted avg       0.99      0.95      0.97      1582
 samples avg       0.65      0.66      0.65      1582



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# Print confusion matrix
# Print confusion matrix
conf_matrix = multilabel_confusion_matrix(y_test, y_pred_tf)
print("Confusion Matrix:")
print(conf_matrix)

#For the 'obsession' class:
#True Positive (TP): 511
#False Positive (FP): 28
#True Negative (TN): 1394
#False Negative (FN): 67

#For the 'compulsion' class:
#True Positive (TP): 968
#False Positive (FP): 35
#True Negative (TN): 961
#False Negative (FN): 36


#True Positive (TP): Instances correctly predicted as belonging to the class.
#False Positive (FP): Instances incorrectly predicted as belonging to the class (they don't actually belong to it).
#True Negative (TN): Instances correctly predicted as not belonging to the class.
#False Negative (FN): Instances incorrectly predicted as not belonging to the class (they actually belong to it).


Confusion Matrix:
[[[1417    5]
  [ 161  417]]

 [[ 910   86]
  [  73  931]]]


## NB using tf-idf

In [ ]:
#intialize the Random Forest Model
clf_NB = MultiOutputClassifier(MultinomialNB()).fit(X_train_tfidf, y_train)
#train the model
prediction_NB_tf = clf_NB.predict(X_test_tfidf)

In [ ]:
label_names = ['obsession','compulsion']
print(classification_report(y_test, prediction_NB_tf,target_names=label_names))

              precision    recall  f1-score   support

   obsession       0.86      0.22      0.35       578
  compulsion       0.72      0.87      0.79      1004

   micro avg       0.73      0.63      0.68      1582
   macro avg       0.79      0.55      0.57      1582
weighted avg       0.77      0.63      0.63      1582
 samples avg       0.47      0.44      0.45      1582



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# Print confusion matrix
# Print confusion matrix
conf_matrix = multilabel_confusion_matrix(y_test, prediction_NB_tf)
print("Confusion Matrix:")
print(conf_matrix)

#For the 'obsession' class:
#True Positive (TP): 511
#False Positive (FP): 28
#True Negative (TN): 1394
#False Negative (FN): 67

#For the 'compulsion' class:
#True Positive (TP): 968
#False Positive (FP): 35
#True Negative (TN): 961
#False Negative (FN): 36


#True Positive (TP): Instances correctly predicted as belonging to the class.
#False Positive (FP): Instances incorrectly predicted as belonging to the class (they don't actually belong to it).
#True Negative (TN): Instances correctly predicted as not belonging to the class.
#False Negative (FN): Instances incorrectly predicted as not belonging to the class (they actually belong to it).


Confusion Matrix:
[[[1402   20]
  [ 450  128]]

 [[ 649  347]
  [ 128  876]]]


In [ ]:
# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], prediction_NB_tf[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')

Accuracy for obsession: 0.765
Accuracy for compulsion: 0.7625


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, prediction_NB_tf)
print(f'Accuracy: {accuracy}')


Accuracy: 0.6215


## KNN + tf-idf

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Initialize the KNN model with specified parameters
knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto')

# Train the KNN model
knn_model.fit(X_train_tfidf, y_train)

# Make predictions on the test set
knn_predictions_tf = knn_model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, knn_predictions_tf)

print(f'Accuracy for KNN: {accuracy}')

Accuracy for KNN: 0.314


In [ ]:
#classification report
label_names = ['obsession','compulsion']
print(classification_report(y_test, knn_predictions_tf,target_names=label_names))

              precision    recall  f1-score   support

   obsession       0.00      0.00      0.00       578
  compulsion       0.75      0.00      0.01      1004

   micro avg       0.75      0.00      0.00      1582
   macro avg       0.38      0.00      0.00      1582
weighted avg       0.48      0.00      0.00      1582
 samples avg       0.00      0.00      0.00      1582



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], knn_predictions_tf[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')

Accuracy for obsession: 0.711
Accuracy for compulsion: 0.499


In [ ]:
conf_matrix = multilabel_confusion_matrix(y_test, knn_predictions_tf)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[[1422    0]
  [ 578    0]]

 [[ 995    1]
  [1001    3]]]


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred_rf_tf= rf_classifier.predict(X_test_tfidf)

# Evaluate the model
conf_matrix = multilabel_confusion_matrix(y_test, y_pred_rf_tf)

# Print confusion matrix for each class
for i, label in enumerate(['obsession', 'compulsion']):
    print(f"Confusion Matrix for {label}:")
    print(conf_matrix[i])
    print("\n")


Confusion Matrix for obsession:
[[1421    1]
 [  97  481]]


Confusion Matrix for compulsion:
[[867 129]
 [ 16 988]]




In [ ]:
#classification report
label_names = ['obsession','compulsion']
print(classification_report(y_test, y_pred_rf_tf,target_names=label_names))

              precision    recall  f1-score   support

   obsession       1.00      0.83      0.91       578
  compulsion       0.88      0.98      0.93      1004

   micro avg       0.92      0.93      0.92      1582
   macro avg       0.94      0.91      0.92      1582
weighted avg       0.93      0.93      0.92      1582
 samples avg       0.63      0.64      0.63      1582



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], y_pred_rf_tf[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')

Accuracy for obsession: 0.951
Accuracy for compulsion: 0.9275


## BOW using labeled data and manual label

In [ ]:
#Data with negative sample (without annotation)
df = pd.read_csv('/content/drive/MyDrive/post classification/processed_data/posts-ontology-manual-label.csv')

In [ ]:
df.head()

In [ ]:
df['post'] = df['post'].astype(str)
df = clean_text(df)
df['post'] = df['post'].apply(process_text)
#process_df['clean_text'] = process_df['clean_text'].apply(process_text)
df.head()

### BOW and LogisticRegression

In [ ]:
# Creating features using Bag of Word
text_data = df['post'].astype(str)

# Convert text to vectors using CountVectorizer
count_vect = CountVectorizer()
X = count_vect.fit_transform(text_data)

# Assuming 'compulsion' and 'obsession' are the target labels
Y = np.asarray(df[['compulsion', 'obsession']])

In [ ]:
#devide the data into train and test
# Get the shape (size) of the DTM
num_documents, num_tokens = X.shape

# Print the number of documents and tokens
print("Number of documents:", num_documents)
print("Number of unique tokens:", num_tokens)
print(X.shape)
print(Y.shape)


Number of documents: 9996
Number of unique tokens: 25399
(9996, 25399)
(9996, 2)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(7996, 25399)
(2000, 25399)
(7996, 2)


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

# Assuming X and Y are your features and labels
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Handle NaN values in features using SimpleImputer
imputer_features = SimpleImputer(strategy='mean')
X_train_imputed = imputer_features.fit_transform(X_train)
X_test_imputed = imputer_features.transform(X_test)

# Handle NaN values in labels using SimpleImputer
imputer_labels = SimpleImputer(strategy='most_frequent')
y_train_imputed = imputer_labels.fit_transform(y_train)
y_test_imputed = imputer_labels.transform(y_test)

# Initialize the logistic regression model
clf_LR = MultiOutputClassifier(LogisticRegression())

# Fit the model to the training data
clf_LR.fit(X_train_imputed, y_train_imputed)

# Make predictions on the test set
predictions = clf_LR.predict(X_test_imputed)

# Evaluate the model
print(classification_report(y_test_imputed, predictions))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

              precision    recall  f1-score   support

           0       0.97      0.93      0.95       499
           1       0.95      0.94      0.94       996

   micro avg       0.95      0.94      0.95      1495
   macro avg       0.96      0.93      0.95      1495
weighted avg       0.96      0.94      0.95      1495
 samples avg       0.61      0.62      0.61      1495



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
